# 🔗 Match Projections to Sleeper Player IDs

This notebook matches players from the **projections database** to the **Sleeper player database** to create a unified table with Sleeper player IDs.

**⚠️ IMPORTANT: Run `post_scraping_processing.ipynb` FIRST!**

This notebook expects cleaned data. Before running this notebook:
1. Run all scrapers to populate data
2. Run `post_scraping_processing.ipynb` to standardize team names (WSH→WAS, JAC→JAX, LA→LAR) and clean positions
3. Then run this notebook to match projections to Sleeper IDs

**Matching Strategy:**
1. Same team
2. Same position
3. Same last name (normalized: lowercase, no special chars, first word only)
4. Same first character of first name
5. Hardcoded exceptions for tricky matches

**Output:**
- New table with Sleeper player IDs added
- List of unmatched projections
- Match statistics and quality report


## 📦 Setup & Imports


In [1]:
import sqlite3
import sys
import re
import pandas as pd
from pathlib import Path

# Add parent directory to path for imports
sys.path.append(str(Path().absolute().parent.parent))
sys.path.append(str(Path().absolute().parent / 'scrapers'))

from backend.scrapers.database import ProjectionsDB
from backend.scrapers.database_league import LeagueDB

print("✓ Imports successful")

# Get absolute paths to databases
NOTEBOOK_DIR = Path().absolute()
BACKEND_DIR = NOTEBOOK_DIR.parent
DB_PROJ_PATH = str(BACKEND_DIR / "data" / "databases" / "projections.db")
DB_LEAGUE_PATH = str(BACKEND_DIR / "data" / "databases" / "league.db")


✓ Imports successful


## 🎯 Hardcoded Name Mappings

Add special cases here where names don't match using standard rules.


In [2]:
# Hardcoded mappings: (projection_first_name, projection_last_name) -> sleeper_player_id
HARDCODED_MATCHES = {
    ("Bam", "Knight"): "6945",  # Zonovan Knight
}

print(f"Loaded {len(HARDCODED_MATCHES)} hardcoded mappings")


Loaded 1 hardcoded mappings


## 🔧 Normalization Functions


In [3]:
def normalize_last_name(name):
    """
    Normalize last name:
    - Lowercase
    - Remove special characters (keep only letters)
    - Take only first word (before space)
    
    Examples:
    - "Tracy Jr." -> "tracy"
    - "St. Brown" -> "st"
    - "O'Brien" -> "obrien"
    """
    if not name:
        return ""
    
    # Convert to lowercase
    name = str(name).lower()
    
    # Take only first word (split on space, take first)
    name = name.split()[0] if name.split() else name
    
    # Remove all non-letter characters
    name = re.sub(r'[^a-z]', '', name)
    
    return name

def get_first_char(name):
    """
    Get first character of first name (lowercase).
    """
    if not name:
        return ""
    name = str(name).strip().lower()
    return name[0] if name else ""

# Test the functions
print("Testing normalization:")
print(f"  'Tracy Jr.' -> '{normalize_last_name('Tracy Jr.')}'")
print(f"  'St. Brown' -> '{normalize_last_name('St. Brown')}'")
print(f"  'Cook III' -> '{normalize_last_name('Cook III')}'")
#print(f"  \'O'Brien\' -> '{normalize_last_name(\"O'Brien\")}'")
print(f"  'All Jr.' -> '{normalize_last_name('All Jr.')}'")
print(f"\nFirst char:")
print(f"  'Christian' -> '{get_first_char('Christian')}'")
print(f"  'C.' -> '{get_first_char('C.')}'")



Testing normalization:
  'Tracy Jr.' -> 'tracy'
  'St. Brown' -> 'st'
  'Cook III' -> 'cook'
  'All Jr.' -> 'all'

First char:
  'Christian' -> 'c'
  'C.' -> 'c'


## 📊 Load Data from Both Databases


In [4]:
# Load projections (cleaned data from post_scraping_processing.ipynb)
print("⚠️  IMPORTANT: Make sure you ran post_scraping_processing.ipynb first!")
print("=" * 70)
print("\nLoading projections from cleaned database...")
with ProjectionsDB(db_path=DB_PROJ_PATH) as db:
    projections = db.get_projections()

print(f"  ✓ Loaded {len(projections)} projection records")

# Load NFL players (cleaned data from post_scraping_processing.ipynb)
print("\nLoading Sleeper NFL players from cleaned database...")
with LeagueDB(db_path=DB_LEAGUE_PATH) as db:
    nfl_players = db.get_nfl_players()

print(f"  ✓ Loaded {len(nfl_players)} NFL players")

# Show sample data
print("\nSample projection:")
if projections:
    sample = projections[0]
    print(f"  {sample.get('player_first_name')} {sample.get('player_last_name')}")
    print(f"  Position: {sample.get('position')}, Team: {sample.get('team')}")
    print(f"  Source: {sample.get('source_website')}, Week: {sample.get('week')}")

print("\nSample NFL player:")
if nfl_players:
    sample = nfl_players[0]
    print(f"  {sample.get('first_name')} {sample.get('last_name')}")
    print(f"  Position: {sample.get('position')}, Team: {sample.get('team')}")
    print(f"  Sleeper ID: {sample.get('player_id')}")
    
print("\n" + "=" * 70)
print("✅ Data loaded successfully! Both databases should already be cleaned.")
print("   If you see mismatches with team names (LA, JAC, WSH),")
print("   run post_scraping_processing.ipynb first!")
print("=" * 70)


⚠️  IMPORTANT: Make sure you ran post_scraping_processing.ipynb first!

Loading projections from cleaned database...
  ✓ Loaded 1625 projection records

Loading Sleeper NFL players from cleaned database...
  ✓ Loaded 3968 NFL players

Sample projection:
  Josh Allen
  Position: QB, Team: BUF
  Source: sleeper.com, Week: Week 10

Sample NFL player:
  A.J. Derby
  Position: TE, Team: None
  Sleeper ID: 2503

✅ Data loaded successfully! Both databases should already be cleaned.
   If you see mismatches with team names (LA, JAC, WSH),
   run post_scraping_processing.ipynb first!


In [5]:
df = pd.DataFrame(nfl_players)
df.to_csv('nfl_players.csv', index=False)
df

,player_id,full_name,first_name,last_name,position,team,number,age,height,weight,...,injury_notes,injury_start_date,practice_participation,depth_chart_position,depth_chart_order,search_rank,fantasy_positions,metadata,created_at,updated_at
0,2503,A.J. Derby,A.J.,Derby,TE,None,81.0,30.0,77,240,...,None,None,None,None,NaN,9999999.0,['TE'],None,2025-11-05 00:47:44,2025-11-05 00:47:44
1,7074,A.J. Hines,A.J.,Hines,RB,None,NaN,NaN,"5'11""",237,...,None,None,None,None,NaN,9999999.0,['RB'],{'override_active': 'false'},2025-11-05 00:47:44,2025-11-05 00:47:44
2,6664,A.J. Ouellette,A.J.,Ouellette,RB,None,1.0,26.0,69,209,...,None,None,None,None,NaN,9999999.0,['RB'],{'channel_id': '1116853288680030208'},2025-11-05 00:47:44,2025-11-05 00:47:44
3,6373,A.J. Richardson,A.J.,Richardson,WR,None,0.0,26.0,72,212,...,None,None,None,None,NaN,9999999.0,['WR'],{'channel_id': '1116853240600723456'},2025-11-05 00:47:44,2025-11-05 00:47:44
4,2701,AJ Cruz,AJ,Cruz,WR,None,81.0,27.0,"5'9""",195,...,None,None,None,None,NaN,9999999.0,['WR'],None,2025-11-05 00:47:44,2025-11-05 00:47:44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3963,7604,Tamorrion Terry,Tamorrion,Terry,WR,None,83.0,26.0,75,207,...,None,None,None,None,NaN,NaN,['WR'],"{'override_active': 'false', 'rookie_year': ''...",2025-11-05 00:47:44,2025-11-05 00:47:44
3964,7518,Taylor Russolino,Taylor,Russolino,K,None,0.0,32.0,71,160,...,None,None,None,None,NaN,NaN,['K'],{'channel_id': '1116853401611665408'},2025-11-05 00:47:44,2025-11-05 00:47:44
3965,7978,Tre Walker,Tre,Walker,WR,None,0.0,NaN,71,180,...,None,None,None,None,NaN,NaN,['WR'],"{'channel_id': '1116853445437947904', 'rookie_...",2025-11-05 00:47:44,2025-11-05 00:47:44
3966,7949,Trevon Grimes,Trevon,Grimes,WR,None,47.0,NaN,76,218,...,None,None,None,None,NaN,NaN,['WR'],"{'channel_id': '1116853440174096384', 'rookie_...",2025-11-05 00:47:44,2025-11-05 00:47:44


## 🔍 Build Matching Index for Fast Lookup


In [6]:
# Build index for Sleeper players: (team, position, normalized_last, first_char) -> player_data
print("Building Sleeper player index...")

sleeper_index = {}

for player in nfl_players:
    team = player.get('team')
    position = player.get('position')
    last_name = player.get('last_name')
    first_name = player.get('first_name')
    
    if not last_name or not first_name:
        continue
    
    # Normalize
    norm_last = normalize_last_name(last_name)
    first_char = get_first_char(first_name)
    
    if not norm_last or not first_char:
        continue
    
    # Create key: (team, position, normalized_last, first_char)
    # Note: team can be None for free agents
    key = (team, position, norm_last, first_char)
    
    # Store in index (handle duplicates by keeping list)
    if key not in sleeper_index:
        sleeper_index[key] = []
    sleeper_index[key].append(player)

print(f"  ✓ Indexed {len(sleeper_index)} unique key combinations")
print(f"  ✓ Total players indexed: {sum(len(v) for v in sleeper_index.values())}")

# Check for duplicate keys (multiple players matching same criteria)
duplicates = {k: v for k, v in sleeper_index.items() if len(v) > 1}
print(f"  ⚠ Found {len(duplicates)} keys with multiple players (will take first match)")


Building Sleeper player index...
  ✓ Indexed 3708 unique key combinations
  ✓ Total players indexed: 3968
  ⚠ Found 150 keys with multiple players (will take first match)


## 🔗 Match Projections to Sleeper IDs


In [7]:
print("Matching projections to Sleeper players...\n")

matched = []
unmatched = []

hardcoded_used = 0
automatic_matched = 0
dst_matched = 0

for proj in projections:
    first_name = proj.get('player_first_name')
    last_name = proj.get('player_last_name')
    team = proj.get('team')
    position = proj.get('position')
    
    sleeper_id = None
    match_method = None
    
    # Special handling for DST - match by team and position only (ignore names)
    if position == 'DST':
        # Find DST for this team in sleeper database
        for player in nfl_players:
            if player.get('position') == 'DST' and player.get('team') == team:
                sleeper_id = player.get('player_id')
                match_method = "dst_team_match"
                dst_matched += 1
                break
    else:
        # Check hardcoded matches first
        hardcode_key = (first_name, last_name)
        if hardcode_key in HARDCODED_MATCHES:
            sleeper_id = HARDCODED_MATCHES[hardcode_key]
            match_method = "hardcoded"
            hardcoded_used += 1
        else:
            # Try automatic matching
            norm_last = normalize_last_name(last_name)
            first_char = get_first_char(first_name)
            
            if norm_last and first_char:
                # Try with team
                key = (team, position, norm_last, first_char)
                
                if key in sleeper_index:
                    # Found match!
                    matched_players = sleeper_index[key]
                    sleeper_id = matched_players[0]['player_id']  # Take first if multiple
                    match_method = "automatic"
                    automatic_matched += 1
    
    # Create matched record
    matched_record = {
        'sleeper_player_id': sleeper_id,
        'match_method': match_method,
        **proj  # Include all original projection fields
    }
    
    if sleeper_id:
        matched.append(matched_record)
    else:
        unmatched.append(matched_record)

print(f"{'='*70}")
print(f"MATCHING COMPLETE")
print(f"{'='*70}")
print(f"Total projections:     {len(projections)}")
print(f"Matched:               {len(matched)} ({len(matched)/len(projections)*100:.1f}%)")
print(f"  - Automatic:         {automatic_matched}")
print(f"  - DST (team-based):  {dst_matched}")
print(f"  - Hardcoded:         {hardcoded_used}")
print(f"Unmatched:             {len(unmatched)} ({len(unmatched)/len(projections)*100:.1f}%)")
print(f"{'='*70}\n")


Matching projections to Sleeper players...

MATCHING COMPLETE
Total projections:     1625
Matched:               1598 (98.3%)
  - Automatic:         1543
  - DST (team-based):  53
  - Hardcoded:         2
Unmatched:             27 (1.7%)



## 📋 View Unmatched Projections


In [8]:
if unmatched:
    print(f"\n{'='*100}")
    print(f"UNMATCHED PROJECTIONS ({len(unmatched)} records)")
    print(f"{'='*100}\n")
    print(f"{'Source':<20} {'Week':<8} {'Name':<25} {'Pos':<5} {'Team':<5} {'Points':<8}")
    print("-" * 100)
    
    for proj in unmatched[:50]:  # Show first 50
        source = (proj.get('source_website') or '')[:18]
        week = (proj.get('week') or '')[:6]
        name = f"{proj.get('player_first_name') or ''} {proj.get('player_last_name') or ''}"[:25]
        pos = proj.get('position') or ''
        team = proj.get('team') or 'N/A'
        pts = proj.get('projected_points') or 0
        
        print(f"{source:<20} {week:<8} {name:<25} {pos:<5} {team:<5} {pts:<8.1f}")
    
    if len(unmatched) > 50:
        print(f"\n... and {len(unmatched) - 50} more unmatched")
    
    print(f"\n{'='*100}\n")
    
    # Export unmatched to CSV for review
    unmatched_df = pd.DataFrame(unmatched)
    unmatched_df.to_csv('../data/csv/unmatched_projections.csv', index=False)
    print("💾 Exported unmatched projections to 'unmatched_projections.csv'")
else:
    print("✅ All projections matched!")



UNMATCHED PROJECTIONS (27 records)

Source               Week     Name                      Pos   Team  Points  
----------------------------------------------------------------------------------------------------
sleeper.com          Week 1   Matthew Wright            K     HOU   7.6     
fantasypros.com      Week 1   Kyle Juszczyk             RB    SF    2.4     
sleeper.com          Week 1   Alec Ingold               RB    MIA   2.0     
fanduel.com          Week 1   Kyle Juszczyk             RB    SF    1.4     
sleeper.com          Week 1   Kyle Juszczyk             RB    SF    1.4     
fantasypros.com      Week 1   Alec Ingold               RB    MIA   1.0     
fanduel.com          Week 1   Alec Ingold               RB    MIA   1.0     
sleeper.com          Week 1   C.J. Ham                  RB    MIN   0.8     
fantasypros.com      Week 1   Patrick Ricard            RB    BAL   0.8     
fantasypros.com      Week 1   Adam Prentice             RB    DEN   0.8     
fantasypros.com

PermissionError: [Errno 13] Permission denied: '../data/csv/unmatched_projections.csv'

## 💾 Save Matched Data to New Table


In [9]:
print("Saving matched projections to database...\n")

# Connect to projections database
conn = sqlite3.connect(DB_PROJ_PATH)
cursor = conn.cursor()

# Create new table with Sleeper ID
cursor.execute("""
    CREATE TABLE IF NOT EXISTS projections_with_sleeper (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        sleeper_player_id TEXT,
        match_method TEXT,
        source_website TEXT,
        week TEXT,
        player_first_name TEXT,
        player_last_name TEXT,
        position TEXT,
        team TEXT,
        projected_points REAL,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        UNIQUE(source_website, week, player_first_name, player_last_name, position)
    )
""")

print("  ✓ Created table 'projections_with_sleeper'")

# Clear existing data
cursor.execute("DELETE FROM projections_with_sleeper")
print("  ✓ Cleared old data")

# Insert all matched records (including unmatched with NULL sleeper_id)
all_records = matched + unmatched

for record in all_records:
    cursor.execute("""
        INSERT OR REPLACE INTO projections_with_sleeper 
        (sleeper_player_id, match_method, source_website, week, 
         player_first_name, player_last_name, position, team, projected_points)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
    """, (
        record.get('sleeper_player_id'),
        record.get('match_method'),
        record.get('source_website'),
        record.get('week'),
        record.get('player_first_name'),
        record.get('player_last_name'),
        record.get('position'),
        record.get('team'),
        record.get('projected_points')
    ))

conn.commit()
print(f"  ✓ Inserted {len(all_records)} records")

# Verify
cursor.execute("SELECT COUNT(*) FROM projections_with_sleeper")
count = cursor.fetchone()[0]
print(f"  ✓ Verified: {count} total records in table")

cursor.execute("SELECT COUNT(*) FROM projections_with_sleeper WHERE sleeper_player_id IS NOT NULL")
matched_count = cursor.fetchone()[0]
print(f"  ✓ Verified: {matched_count} records with Sleeper ID")

conn.close()

print("\n✅ Matched projections saved to 'projections_with_sleeper' table!")


Saving matched projections to database...

  ✓ Created table 'projections_with_sleeper'
  ✓ Cleared old data
  ✓ Inserted 1625 records
  ✓ Verified: 1625 total records in table
  ✓ Verified: 1598 records with Sleeper ID

✅ Matched projections saved to 'projections_with_sleeper' table!


## 📊 View Sample of Matched Data


In [10]:
# Query and display sample data
conn = sqlite3.connect(DB_PROJ_PATH)

# Get sample of matched records
query = """
    SELECT 
        sleeper_player_id,
        player_first_name || ' ' || player_last_name as player_name,
        position,
        team,
        source_website,
        week,
        projected_points,
        match_method
    FROM projections_with_sleeper
    WHERE sleeper_player_id IS NOT NULL
    ORDER BY projected_points DESC
    LIMIT 25
"""

df = pd.read_sql_query(query, conn)

print(f"\n{'='*100}")
print(f"TOP 25 MATCHED PROJECTIONS (by points)")
print(f"{'='*100}\n")
print(df.to_string(index=False))
print(f"\n{'='*100}\n")

conn.close()



TOP 25 MATCHED PROJECTIONS (by points)

sleeper_player_id         player_name position team   source_website    week  projected_points match_method
             4984          Josh Allen       QB  BUF      sleeper.com Week 10             25.60    automatic
             4881       Lamar Jackson       QB  BAL      sleeper.com Week 10             25.10    automatic
             4034 Christian McCaffrey       RB   SF         espn.com Week 10             24.40    automatic
             4034 Christian McCaffrey       RB   SF      sleeper.com Week 10             24.20    automatic
            12508         Jaxson Dart       QB  NYG      sleeper.com Week 10             24.10    automatic
             4984          Josh Allen       QB  BUF  fantasypros.com Week 10             24.10    automatic
             9493          Puka Nacua       WR  LAR      sleeper.com Week 10             23.70    automatic
             4881       Lamar Jackson       QB  BAL         espn.com Week 10             23.60 

## 📈 Match Quality Analysis


In [11]:
conn = sqlite3.connect(DB_PROJ_PATH)

print(f"\n{'='*70}")
print(f"MATCH QUALITY REPORT")
print(f"{'='*70}\n")

# By source
query = """
    SELECT 
        source_website,
        COUNT(*) as total,
        SUM(CASE WHEN sleeper_player_id IS NOT NULL THEN 1 ELSE 0 END) as matched,
        ROUND(100.0 * SUM(CASE WHEN sleeper_player_id IS NOT NULL THEN 1 ELSE 0 END) / COUNT(*), 1) as match_pct
    FROM projections_with_sleeper
    GROUP BY source_website
    ORDER BY match_pct DESC
"""

df_sources = pd.read_sql_query(query, conn)
print("Match Rate by Source:")
print(df_sources.to_string(index=False))

# By position
print("\nMatch Rate by Position:")
query = """
    SELECT 
        position,
        COUNT(*) as total,
        SUM(CASE WHEN sleeper_player_id IS NOT NULL THEN 1 ELSE 0 END) as matched,
        ROUND(100.0 * SUM(CASE WHEN sleeper_player_id IS NOT NULL THEN 1 ELSE 0 END) / COUNT(*), 1) as match_pct
    FROM projections_with_sleeper
    GROUP BY position
    ORDER BY match_pct DESC
"""

df_positions = pd.read_sql_query(query, conn)
print(df_positions.to_string(index=False))

# By week
print("\nMatch Rate by Week:")
query = """
    SELECT 
        week,
        COUNT(*) as total,
        SUM(CASE WHEN sleeper_player_id IS NOT NULL THEN 1 ELSE 0 END) as matched,
        ROUND(100.0 * SUM(CASE WHEN sleeper_player_id IS NOT NULL THEN 1 ELSE 0 END) / COUNT(*), 1) as match_pct
    FROM projections_with_sleeper
    GROUP BY week
    ORDER BY week
"""

df_weeks = pd.read_sql_query(query, conn)
print(df_weeks.to_string(index=False))

print(f"\n{'='*70}\n")

conn.close()



MATCH QUALITY REPORT

Match Rate by Source:
  source_website  total  matched  match_pct
        espn.com    232      232      100.0
firstdown.studio    161      161      100.0
 fantasypros.com    388      383       98.7
     sleeper.com    371      364       98.1
     fanduel.com    473      458       96.8

Match Rate by Position:
position  total  matched  match_pct
      QB    204      204      100.0
     DST     53       53      100.0
      WR    597      593       99.3
      TE    283      278       98.2
       K     81       79       97.5
      RB    407      391       96.1

Match Rate by Week:
   week  total  matched  match_pct
Week 10   1625     1598       98.3




## 💾 Export to CSV for Troubleshooting

Export the complete `projections_with_sleeper` table to CSV for inspection.


In [12]:
print("Exporting projections_with_sleeper table to CSV...\n")

conn = sqlite3.connect(DB_PROJ_PATH)

# Read entire table
query = """
    SELECT 
        sleeper_player_id,
        match_method,
        source_website,
        week,
        player_first_name,
        player_last_name,
        position,
        team,
        projected_points,
        created_at
    FROM projections_with_sleeper
    ORDER BY projected_points DESC
"""

df_export = pd.read_sql_query(query, conn)
conn.close()

# Export to CSV
output_file = '../data/csv/projections_with_sleeper.csv'
df_export.to_csv(output_file, index=False)

print(f"✅ Exported {len(df_export):,} records to '{output_file}'")
print(f"\nBreakdown:")
print(f"  Records with Sleeper ID: {df_export['sleeper_player_id'].notna().sum():,}")
print(f"  Records without Sleeper ID: {df_export['sleeper_player_id'].isna().sum():,}")
print(f"  Unique players (with ID): {df_export[df_export['sleeper_player_id'].notna()]['sleeper_player_id'].nunique()}")
print(f"  Sources: {df_export['source_website'].nunique()}")
print(f"  Weeks: {df_export['week'].nunique()}")

print(f"\n📁 File saved: {output_file}")


Exporting projections_with_sleeper table to CSV...

✅ Exported 1,625 records to '../data/csv/projections_with_sleeper.csv'

Breakdown:
  Records with Sleeper ID: 1,598
  Records without Sleeper ID: 27
  Unique players (with ID): 537
  Sources: 5
  Weeks: 1

📁 File saved: ../data/csv/projections_with_sleeper.csv


## 📝 Summary & Usage Guide

**What was created:**
- New table: `projections_with_sleeper` in `projections.db`
- Columns include: `sleeper_player_id`, `match_method`, plus all original projection fields

**To add hardcoded matches:**
1. Find the unmatched player in the unmatched list above
2. Find their Sleeper ID using `view_player_table.py` or the league database
3. Add to `HARDCODED_MATCHES` dictionary in cell 4
4. Re-run the notebook

**Example:**
```python
HARDCODED_MATCHES = {
    ("Bam", "Knight"): "6945",  # Zonovan Knight
    ("AJ", "Brown"): "6786",     # A.J. Brown
    # Add more as needed
}
```

**Query examples:**
```sql
-- Get all projections for a player
SELECT * FROM projections_with_sleeper 
WHERE sleeper_player_id = '4046';

-- Compare sources for top QBs
SELECT player_first_name, player_last_name, source_website, projected_points
FROM projections_with_sleeper
WHERE position = 'QB' AND week = 'Week 9'
ORDER BY projected_points DESC;
```
